## <span style="color:green"> Notebook - Melt Estimation</span>
<span style="color:purple">Sinan Ozaydin, School of Geosciences, The University of Sydney, NSW 2006, Australia <br/> </span>
<span style="color:purple">sinan.ozaydin@protonmail.com | sinan.ozaydin@sydney.edu.au </span>

In this notebook, we will use experimental electrical conductivity measurements.

In [1]:
#Importing the neccesarry python libraries.
import os
import pide
from pide.inversion import conductivity_solver_single_param, conductivity_metropolis_hastings_two_param
import numpy as np
import matplotlib.pyplot as plt

/home/sinan/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


The estimation of melt content from the given electrical conductivity can be solved with pide for a single parameter. The relationships in melt estimation. Let's first define the environment for the conductivity.

In [4]:
#creating a pide object
p_obj = pide.pide()

#setting up and environment at 1300 K and 3 GPa 
temperature = np.array([1500])
pressure = np.array([3.0])

p_obj.set_temperature(temperature)
p_obj.set_pressure(pressure)

#setting up a simple lherzolite composition
p_obj.set_composition_solid_mineral(ol = 0.6,opx = 0.3, cpx = 0.05, garnet = 0.05)
#setting the mineral conductivity choices
p_obj.set_mineral_conductivity_choice(ol = 4, opx = 0, garnet = 0, cpx = 6)
p_obj.set_solid_phs_mix_method(1) #HS lower bound

#Setting up Sifre2014 as the conductivity choice
p_obj.set_melt_fluid_conductivity_choice(melt = 0)

#Setting up melt CO2
p_obj.set_melt_properties(co2 = 100)

#Setting up bulk water.
#Try to avoid setting up melt_h2o from set_melt properties, because this will not distribute water among minerals.
p_obj.set_bulk_water(300)
p_obj.mantle_water_distribute()

#Setting up the melt-solid mixture relationship to Tubes model
p_obj.set_solid_melt_fluid_mix_method(1)

#Setting up the conductivity value to estimate melt for
cond_external = [1e-1] #or 10 ohm m

#Now setting up the conductivity_solver_single_param
#we are using the parameter melt_fluid_mass_frac to estimate instead volumetric parameter melt_frac.
#This allows us to calculate the thermal expansion and water partitioning among melt and solid minerla mixture
#with more accuracy.
melt_fraction_estimate, residual_list = conductivity_solver_single_param(object = p_obj, cond_list = cond_external,
param_name = 'melt_fluid_mass_frac', upper_limit_list = np.ones(len(temperature)), lower_limit_list= np.zeros(len(temperature)),
search_start = 0.01, acceptence_threshold = 0.5, num_cpu = 1)

print(f'Melt Fraction Estimate: {melt_fraction_estimate[0]}')

Melt Fraction Estimate: 0.2625


Melt fraction can be estimated for the given composition of the environment with a *conductivity_solver_single_param* function. Many of the 